# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [23]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster_response_pipeline.db')
df = pd.read_sql_table('BestData', con = engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
def model_accuracy(y_pred, y_test):
    for i, col in enumerate(y_test):
        print('------------------------- \n Column_Name: {}\n'.format(col))
        print(classification_report(y_test[col], y_pred[:, i]))

In [11]:
score = model_accuracy(y_pred, y_test)
score

------------------------- 
 Column_Name: related

             precision    recall  f1-score   support

          0       0.64      0.36      0.46      1577
          1       0.82      0.93      0.87      4927
          2       0.47      0.14      0.22        50

avg / total       0.77      0.79      0.77      6554

------------------------- 
 Column_Name: request

             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5412
          1       0.81      0.37      0.51      1142

avg / total       0.87      0.88      0.86      6554

------------------------- 
 Column_Name: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

------------------------- 
 Column_Name: aid_related

             precision    recall  f1-score   support

          0       0.71      0.88      0.79    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fb49f420488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [13]:
parameters =  {'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [5, 25]
              } 

cv = GridSearchCV(pipeline, param_grid = parameters)

In [14]:
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [5, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [5, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
y_pred = cv.predict(X_test)
score = model_accuracy(y_pred, y_test)

------------------------- 
 Column_Name: related

             precision    recall  f1-score   support

          0       0.71      0.28      0.40      1577
          1       0.80      0.96      0.87      4927
          2       0.67      0.04      0.08        50

avg / total       0.78      0.79      0.75      6554

------------------------- 
 Column_Name: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.94      5412
          1       0.86      0.43      0.58      1142

avg / total       0.89      0.89      0.87      6554

------------------------- 
 Column_Name: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

------------------------- 
 Column_Name: aid_related

             precision    recall  f1-score   support

          0       0.75      0.87      0.80    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
pipeline_mod = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])
pipeline_mod.fit(X_train, y_train)
y_pred = pipeline_mod.predict(X_test)
score = model_accuracy(y_pred, y_test)
score

------------------------- 
 Column_Name: related

             precision    recall  f1-score   support

          0       0.59      0.24      0.34      1577
          1       0.79      0.94      0.86      4927
          2       0.26      0.12      0.16        50

avg / total       0.74      0.77      0.73      6554

------------------------- 
 Column_Name: request

             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5412
          1       0.79      0.54      0.64      1142

avg / total       0.89      0.89      0.89      6554

------------------------- 
 Column_Name: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.14      0.03      0.04        38

avg / total       0.99      0.99      0.99      6554

------------------------- 
 Column_Name: aid_related

             precision    recall  f1-score   support

          0       0.74      0.87      0.80    

In [18]:
pipeline_mod.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fb49f420488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [19]:
parameters_mod = {
    'clf__estimator__learning_rate': [0.3, 0.5, 0.75]
}

In [20]:
cv_mod = GridSearchCV(pipeline_mod, param_grid = parameters_mod)

In [21]:
cv_mod.fit(X_train, y_train)
y_pred = cv_mod.predict(X_test)
score = model_accuracy(y_pred, y_test)
score

------------------------- 
 Column_Name: related

             precision    recall  f1-score   support

          0       0.60      0.16      0.25      1577
          1       0.78      0.97      0.86      4927
          2       0.50      0.06      0.11        50

avg / total       0.73      0.77      0.71      6554

------------------------- 
 Column_Name: request

             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5412
          1       0.81      0.54      0.65      1142

avg / total       0.89      0.90      0.89      6554

------------------------- 
 Column_Name: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

------------------------- 
 Column_Name: aid_related

             precision    recall  f1-score   support

          0       0.74      0.87      0.80    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [25]:
pickle.dump(cv_mod, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.